In [ ]:
!pip install keras
!pip install numpy
!pip install pandas
!pip install tensorflow
!pip install lime

In [ ]:
import shutil
import random
import numpy as np
import json
import zipfile
import os
import pandas as pd
import PIL
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import os
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import lime
from lime import lime_image
from PIL import Image

In [ ]:
!wget "http://seppe.net/aa/assignment2/images.zip"
!7z x images.zip

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import os
images_path = '/content/images.zip'
dataset_path = '/content/gdrive/My Drive/dataset.json'

Extract the price categories and store them in a library where they are linked to images. (We will later need this as a dataframe)

In [ ]:
with open(dataset_path, "r") as f:
    data = json.load(f)
price_categories = {}
image_ids = []
for item in data:
    # Check if 'price_category' key is present in the 'item' dictionary
    if 'price_category' in item and item['price_category'] is not None:
        # Extract the price category
        price_category = item['price_category']['label']
        # Extract the image IDs from the "full_images" list and map them to their price category
        for img in item["more_details"]["full_images"]:
            image_id = img["image_id"]
            image_ids.append(image_id)
            price_categories[image_id] = price_category


Now I have a list of image id's and a library of price categories.
The file names of the images are just the id with .jpg behind it.
I want to create the training, validation and test set.
For this I need to make a pandas data frame with two columns, with the image file names and one with the categories.

First, we shuffle the image id's and then split them up in training, validation and test set lists of image id's.(The training, validation, test sizes are 70%, 20% and 10%. We could, however **change these values 0.7, 0.2, 0.1 to lower values if we want to use less of the data**.)

In [ ]:
random.shuffle(image_ids)  # shuffle image id's

train_size = 0.07
val_size = 0.02
test_size = 0.01

train_image_ids = image_ids[:int(len(image_ids) * train_size)]
val_image_ids = image_ids[int(len(image_ids) * train_size):int(len(image_ids) * (train_size + val_size))]
test_image_ids = image_ids[int(len(image_ids) * (train_size + val_size)):]

Now we create lists of the filenames of these images.

In [ ]:
train_filenames = [f"{image_id}.jpg" for image_id in train_image_ids]
val_filenames = [f"{image_id}.jpg" for image_id in val_image_ids]
test_filenames = [f"{image_id}.jpg" for image_id in test_image_ids]


Now, we create a pandas dataframe with two columns: "filename" and "price_category".

In [ ]:
train_df = pd.DataFrame(
    {"filename": train_filenames, "price_category": [price_categories[image_id] for image_id in train_image_ids]})
val_df = pd.DataFrame(
    {"filename": val_filenames, "price_category": [price_categories[image_id] for image_id in val_image_ids]})
test_df = pd.DataFrame(
    {"filename": test_filenames, "price_category": [price_categories[image_id] for image_id in test_image_ids]})

We set the paths for the training, validation, and test data directories and create the directories.

We set the paths for the training, validation, and test data directories and create the directories.

In [ ]:
# Set the paths for the training, validation, and test data directories
train_data_dir = os.path.join(images_path, "train")
val_data_dir = os.path.join(images_path, "val")
test_data_dir = os.path.join(images_path, "test")

# Create the directories if they don't exist already
!mkdir -p "{train_data_dir}"
!mkdir -p "{val_data_dir}"
!mkdir -p "{test_data_dir}"

In [ ]:
import zipfile

zip_path = '/content/images.zip'
target_dir = '/content/images'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_dir)

We copy the image files to the appropriate directories.

In [ ]:
for filename in train_filenames:
    src_path = os.path.join(target_dir, filename)
    dst_path = os.path.join(train_data_dir, filename)
    shutil.copy(src_path, dst_path)

for filename in val_filenames:
    src_path = os.path.join(target_dir, filename)
    dst_path = os.path.join(val_data_dir, filename)
    shutil.copy(src_path, dst_path)

for filename in test_filenames:
    src_path = os.path.join(target_dir, filename)
    dst_path = os.path.join(test_data_dir, filename)
    shutil.copy(src_path, dst_path)

Now we want to preprocess the data and then generate it.


We first define the image dimensions and the batch size. **(Change this, see what works best)**

In [ ]:
img_width, img_height = 224, 224
batch_size = 32

We use ImageDataGenerator for preprocessing including rotation augmentation.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=20, fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

We generate the training, validation, and testing datasets.

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
        '/content/images/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        '/content/images/val',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        '/content/images/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

Found 105071 images belonging to 1 classes.
Found 105071 images belonging to 1 classes.
Found 105071 images belonging to 1 classes.


Now we can finally build our Neural Network. We created one by hand at first, and then used a pretrained model (VGG16) to compare the results.

Define the CNN structure.

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

Compile the model.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model.(**Number of epochs can be changed**)

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=50,
                              validation_data=val_generator, validation_steps=val_generator.samples // batch_size)

Evaluate the model.

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)


Interpreter LIME


In [ ]:
explainer = lime_image.LimeImageExplainer()


def predict_wrapper(images):
    # This function is a wrapper around the model's prediction function
    # It takes in a batch of images (N, 224, 224, 3) and returns the predicted probabilities (N, 5)
    return model.predict(images)


# Get an example image from the validation set
example_image_path = os.path.join(val_data_dir, val_filenames[0])
example_image = Image.open(example_image_path)

# Explain the model's prediction for the example image
explanation = explainer.explain_instance(np.array(example_image), predict_wrapper, top_labels=5)

# Show the LIME visualization for the explanation
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
img_boundry = mark_boundaries(temp/2 + 0.5, mask)
Image.fromarray((img_boundry*255).astype(np.uint8))

Now, we do the same thing but with the pretrained model VGG16.

We first preprocess and generate.

In [ ]:
# preprocessing according to VGG16
batch_size = 64
train_datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=90,
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5,
                                     height_shift_range=0.5,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(rescale=1. / 255,preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(rescale=1. / 255,preprocessing_function=preprocess_input)

# Generate the training, validation, and testing datasets (I followed the instructions on https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/)
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height),
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True,
                                               seed=42)
val_generator = val_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height),
                                                batch_size=batch_size, class_mode='categorical', shuffle=True,
                                               seed=42)
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                             target_size=(img_width, img_height),
                                             class_mode='categorical',
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)


NameError: ignored

Load without the top layer input_shape(img_width, img_height, 3) (the 3 is for the colors rbg).

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg_model.layers:
    layer.trainable = False
    # This is to make sure we don't train these layers

We add a new output layer with 4 categories. Then we create the new model with the pretrained layers and our new output layer.

In [ ]:
x = Flatten()(vgg_model.output)
output_layer = Dense(4, activation='softmax')(x)

model_VGG = Model(inputs=vgg_model.input, outputs=output_layer)

Compile the model.

In [ ]:
model_VGG.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model. (**Number of epochs can be changed**)

In [ ]:
history = model_VGG.fit_generator(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=5,
                              validation_data=val_generator, validation_steps=val_generator.samples // batch_size)

Evaluate the model.


In [ ]:
test_loss, test_acc = model_VGG.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

LIME interpreter

In [ ]:
explainer = lime_image.LimeImageExplainer()


def predict_wrapper_VGG(images):
    # This function is a wrapper around the model's prediction function
    # It takes in a batch of images (N, 224, 224, 3) and returns the predicted probabilities (N, 5)
    return model_VGG.predict(images)


# Get an example image from the validation set
example_image_path = os.path.join(val_data_dir, val_filenames[0])
example_image = Image.open(example_image_path)

# Explain the model's prediction for the example image
explanation = explainer.explain_instance(np.array(example_image), predict_wrapper_VGG, top_labels=5)

# Show the LIME visualization for the explanation
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
img_boundry = mark_boundaries(temp/2 + 0.5, mask)
Image.fromarray((img_boundry*255).astype(np.uint8))